In [1]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta
import pandas as pd

In [14]:
# Load the dataset#
file_path = r'C:\Users\tanap\OneDrive\เดสก์ท็อป\MADT3-NIDA\(3-2566) MADT 8101 Seminar in Advanced Analytics\hackathon\Merged_Transactions_Data_V002.csv'
data = pd.read_csv(file_path)
data.head()

,customer_id,transaction_date,product_id,amount,name,gender,age,telephone,email,product_name,product_description,year,month,day,weekday
0,1,2023-01-01,14,126,Jane Jones,Female,61,555-2082211,user0@gmail.com,Organic Chicken,Free-range and tasty.,2023,1,1,6
1,1,2023-01-15,14,163,Jane Jones,Female,61,555-2082211,user0@gmail.com,Organic Chicken,Free-range and tasty.,2023,1,15,6
2,1,2023-02-25,1,180,Jane Jones,Female,61,555-2082211,user0@gmail.com,Organic Apples,Fresh and organic.,2023,2,25,5
3,1,2023-04-04,14,115,Jane Jones,Female,61,555-2082211,user0@gmail.com,Organic Chicken,Free-range and tasty.,2023,4,4,1
4,1,2023-06-08,19,77,Jane Jones,Female,61,555-2082211,user0@gmail.com,Organic Eggs,Farm-fresh and wholesome.,2023,6,8,3


In [18]:
# Convert transaction_date to datetime#####
data['transaction_date'] = pd.to_datetime(data['transaction_date'])

# Demographic features
demographic_features = data[['customer_id', 'gender', 'age', 'telephone', 'email']].drop_duplicates()
demographic_features['male'] = np.where(demographic_features['gender'] == 'Male', 1, 0)
demographic_features['female'] = np.where(demographic_features['gender'] == 'Female', 1, 0)

# Transaction features
transaction_features = data.groupby('customer_id').agg(
    total_amount_spent=('amount', 'sum'),
    average_transaction_amount=('amount', 'mean'),
    number_of_transactions=('amount', 'count'),
    first_transaction_date=('transaction_date', 'min'),
    last_transaction_date=('transaction_date', 'max')
).reset_index()

# Calculate recency
transaction_features['recency'] = (pd.Timestamp('now') - transaction_features['last_transaction_date']).dt.days

# Calculate frequency of transactions (per month)
transaction_features['transaction_frequency_per_month'] = transaction_features['number_of_transactions'] / (
    (transaction_features['last_transaction_date'] - transaction_features['first_transaction_date']).dt.days / 30
)

# Calculate tenure (days since first transaction)
transaction_features['tenure'] = (pd.Timestamp('now') - transaction_features['first_transaction_date']).dt.days

# Most frequently purchased product
most_frequent_product = data.groupby(['customer_id', 'product_name']).size().reset_index(name='counts')
most_frequent_product = most_frequent_product.loc[most_frequent_product.groupby('customer_id')['counts'].idxmax()]
most_frequent_product = most_frequent_product[['customer_id', 'product_name']].rename(columns={'product_name': 'most_frequent_product'})

# Total amount spent per product category
total_spent_per_product = data.groupby(['customer_id', 'product_name'])['amount'].sum().unstack(fill_value=0).reset_index()

# Time-based features
data['year_month'] = data['transaction_date'].dt.to_period('M')
time_based_features = data.groupby(['customer_id', 'year_month'])['amount'].sum().reset_index()
average_spent_per_month = time_based_features.groupby('customer_id')['amount'].mean().reset_index().rename(columns={'amount': 'average_spent_per_month'})

# Day of the week preference
day_of_week_preference = data.groupby(['customer_id', 'weekday']).size().unstack(fill_value=0).reset_index()
day_of_week_preference['preferred_day_of_week'] = day_of_week_preference.drop('customer_id', axis=1).idxmax(axis=1)

# Diversity of products purchased
diversity_of_products = data.groupby('customer_id')['product_id'].nunique().reset_index().rename(columns={'product_id': 'unique_products_purchased'})

# Total number of products purchased
total_products_purchased = data.groupby('customer_id')['amount'].count().reset_index().rename(columns={'amount': 'total_products_purchased'})

# Calculate aggregated features for the last 1, 3, 6, and 12 months
now = pd.Timestamp('now')
time_periods = {
    '1_month': 30,
    '3_months': 90,
    '6_months': 180,
    '12_months': 365
}

for period_name, days in time_periods.items():
    mask = (now - data['transaction_date']).dt.days <= days
    period_data = data[mask].groupby('customer_id')['amount'].agg(
        **{f'total_amount_spent_{period_name}': 'sum', 
           f'average_transaction_amount_{period_name}': 'mean', 
           f'number_of_transactions_{period_name}': 'count'}
    ).reset_index()
    transaction_features = transaction_features.merge(period_data, on='customer_id', how='left').fillna(0)

# Calculate product category features
product_category_features = data.groupby(['customer_id', 'product_name']).agg(
    total_spent_per_category=('amount', 'sum'),
    transaction_count_per_category=('amount', 'count')
).unstack(fill_value=0).reset_index()

product_category_features.columns = ['_'.join(col).strip() if type(col) is tuple else col for col in product_category_features.columns.values]
product_category_features['customer_id'] = product_category_features['customer_id_']
product_category_features = product_category_features.drop(columns='customer_id_', axis=1)
product_category_features.columns = [col.lower() for col in product_category_features.columns]
product_category_features.columns = ['product_' + col if col != 'customer_id' else col for col in product_category_features.columns]

# Sentiment
file_path_sentiment = r'C:\Users\tanap\OneDrive\เดสก์ท็อป\MADT3-NIDA\(3-2566) MADT 8101 Seminar in Advanced Analytics\hackathon\group2_data\group2_customer_sentiment.csv'
data_sentiment = pd.read_csv(file_path_sentiment)

# Topic Modeling
file_path_group2_topic_modeling = r'C:\Users\tanap\OneDrive\เดสก์ท็อป\MADT3-NIDA\(3-2566) MADT 8101 Seminar in Advanced Analytics\hackathon\group2_data\group2_topic_modeling.csv'
data_topic_modeling = pd.read_csv(file_path_group2_topic_modeling)
data_topic_modeling = data_topic_modeling.pivot_table(index='Customer ID', columns='Intent', aggfunc='size', fill_value=0)
topic_name = ['Dissatisfaction with Prices', 'Positive Feedback on Product Selection', 'Loyalty', 'Positive Feedback on Store Environment', 
 'Satisfaction with Specific Product Categories',' Negative Feedback on Service', 'Switching to Competitors', 'Recommendations for Improvement', 
 'Interest in Promotions or Discounts', 'Convenience', 'Product Availability', 'Customer Support', 'Product Quality', 'Store Location', 
 'General Feedback']

# Ensure all specified categories are present in the pivot table
for category in topic_name:
    if category not in data_topic_modeling.columns:
        data_topic_modeling[category] = 0

topic_name_rename = data_topic_modeling.columns

data_topic_modeling = data_topic_modeling[topic_name_rename].reset_index()
data_topic_modeling.columns = [col.lower() for col in data_topic_modeling.columns]
data_topic_modeling.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
data_topic_modeling.columns = ['topic_modeling_' + col if col != 'customer_id' else col for col in data_topic_modeling.columns]

# product cat
file_path_product_cat = r'C:\Users\tanap\OneDrive\เดสก์ท็อป\MADT3-NIDA\(3-2566) MADT 8101 Seminar in Advanced Analytics\hackathon\group2_data\group2_product_category.csv'
data_product_cat = pd.read_csv(file_path_product_cat)

#result from K-Mean
file_path_result = r'C:\Users\tanap\OneDrive\เดสก์ท็อป\MADT3-NIDA\(3-2566) MADT 8101 Seminar in Advanced Analytics\hackathon\customer_segmentation_results.csv'
data_result = pd.read_csv(file_path_result)
data_result_key_result = data_result[['customer_id', 'segment']]
data_result_age = data_result.filter(regex='age_range')
data_result = pd.concat([data_result_age, data_result_key_result], axis=1)

# Merge all features into a single DataFrame
customer_single_view = demographic_features.merge(transaction_features, on='customer_id')
customer_single_view = customer_single_view.merge(most_frequent_product, on='customer_id', how='left')
customer_single_view = customer_single_view.merge(total_spent_per_product, on='customer_id', how='left')
customer_single_view = customer_single_view.merge(average_spent_per_month, on='customer_id', how='left')
customer_single_view = customer_single_view.merge(day_of_week_preference[['customer_id', 'preferred_day_of_week']], on='customer_id', how='left')
customer_single_view = customer_single_view.merge(diversity_of_products, on='customer_id', how='left')
customer_single_view = customer_single_view.merge(total_products_purchased, on='customer_id', how='left')
customer_single_view = customer_single_view.merge(product_category_features, on='customer_id', how='left')
customer_single_view = customer_single_view.merge(data_sentiment, on='customer_id', how='left')
customer_single_view = customer_single_view.merge(data_topic_modeling, on='customer_id', how='left')
customer_single_view = customer_single_view.merge(data_product_cat, on='customer_id', how='left')
customer_single_view = customer_single_view.merge(data_result, on='customer_id', how='left')

# Display the final DataFrame
print(customer_single_view.shape)
customer_single_view.head()

customer_single_view_cleaned = customer_single_view.drop(columns=['telephone', 'email', 'first_transaction_date', 'last_transaction_date'])
# customer_single_view_cleaned['male'] = np.where(customer_single_view_cleaned['gender'] == 'Male', 1, 0)
# customer_single_view_cleaned['female'] = np.where(customer_single_view_cleaned['gender'] == 'Female', 1, 0)
customer_single_view_cleaned = customer_single_view_cleaned.drop(columns='gender', axis=1)
customer_single_view_cleaned.columns = [col.lower() for col in customer_single_view_cleaned.columns]
customer_single_view_cleaned.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
#customer_single_view_cleaned.select_dtypes(include=['object'])
customer_single_view_cleaned = customer_single_view_cleaned.drop(columns=['most_frequent_product', 'most_frequent_sentiment'])
customer_single_view_cleaned['preferred_day_of_week'] = customer_single_view_cleaned['preferred_day_of_week'].astype('int')
customer_single_view_cleaned


(200, 110)


,customer_id,age,male,female,total_amount_spent,average_transaction_amount,number_of_transactions,recency,transaction_frequency_per_month,tenure,...,topic_modeling_store_location,total_spent_per_category_household,transaction_count_per_category_food_&_fresh_drink,total_spent_per_category_food_&_fresh_drink,transaction_count_per_category_children,age_range_0-25,age_range_26-35,age_range_36-45,age_range_46+,segment
0,1,61,0,1,1280,128.000000,10,29,0.571429,554,...,0,0,10,1280,0,0,0,0,1,2
1,2,33,0,1,1040,86.666667,12,101,0.861244,519,...,0,0,0,0,0,0,1,0,0,0
2,3,29,0,1,471,94.200000,5,165,0.537634,444,...,0,471,0,0,0,0,1,0,0,0
3,4,53,1,0,482,120.500000,4,132,0.944882,259,...,0,0,0,0,4,0,0,0,1,0
4,5,30,0,1,1471,98.066667,15,53,0.943396,530,...,0,0,0,0,15,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,70,0,1,499,124.750000,4,61,0.250522,540,...,0,0,0,0,4,0,0,0,1,0
196,197,19,1,0,1138,75.866667,15,19,0.925926,505,...,0,1138,0,0,0,1,0,0,0,3
197,198,70,0,1,219,109.500000,2,196,1.333333,241,...,0,0,0,0,2,0,0,0,1,0
198,199,46,0,1,1057,105.700000,10,50,0.660793,504,...,0,0,0,0,0,0,0,0,1,0


<div style="text-align: center;"> 
    <p style="font-weight: bold; color: White;"> Save data </p>
    <p style="color: Gray;"> Customer single view </p>
</div>

In [9]:
#save for customer single view
# customer_single_view_cleaned.to_csv(r'C:\Users\tanap\OneDrive\เดสก์ท็อป\MADT3-NIDA\(3-2566) MADT 8101 Seminar in Advanced Analytics\hackathon\group2_data\group2_customer_single_view.csv', index=False, encoding='utf-8')

<div style="text-align: center;"> 
    <p style="font-weight: bold; color: White;"> Save data </p>
    <p style="color: Gray;"> Collaborative filtering </p>
</div>

In [5]:
customer_single_view_cleaned_collaborative_filtering_key = customer_single_view_cleaned[['customer_id', 'segment']]
customer_single_view_cleaned_collaborative_filtering_product = customer_single_view_cleaned.filter(regex='^product_transaction_count')
customer_single_view_cleaned_collaborative_filtering = pd.concat([customer_single_view_cleaned_collaborative_filtering_key, customer_single_view_cleaned_collaborative_filtering_product], axis=1)
customer_single_view_cleaned_collaborative_filtering

,customer_id,segment,product_transaction_count_per_category_baby_diapers,product_transaction_count_per_category_beer,product_transaction_count_per_category_children’s_cereal,product_transaction_count_per_category_cooking_oil,product_transaction_count_per_category_fresh_vegetables,product_transaction_count_per_category_house_cleaning_spray,product_transaction_count_per_category_juice_boxes,product_transaction_count_per_category_laundry_detergent,product_transaction_count_per_category_organic_apples,product_transaction_count_per_category_organic_chicken,product_transaction_count_per_category_organic_eggs,product_transaction_count_per_category_organic_milk,product_transaction_count_per_category_red_wine,product_transaction_count_per_category_toothpaste,product_transaction_count_per_category_whole_wheat_bread,product_transaction_count_per_category_wine_glasses
0,1,2,0,0,0,0,0,0,0,0,2,3,2,3,0,0,0,0
1,2,0,0,2,0,0,0,0,0,0,0,0,0,0,6,0,0,4
2,3,0,0,0,0,0,0,3,0,1,0,0,0,0,0,1,0,0
3,4,0,1,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0
4,5,1,4,0,6,0,0,0,5,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,0,1,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0
196,197,3,0,0,0,0,0,6,0,6,0,0,0,0,0,3,0,0
197,198,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
198,199,0,0,4,0,0,0,0,0,0,0,0,0,0,4,0,0,2


In [ ]:
#save for collaborative filtering
# customer_single_view_cleaned_collaborative_filtering.to_csv(r'C:\Users\tanap\OneDrive\เดสก์ท็อป\MADT3-NIDA\(3-2566) MADT 8101 Seminar in Advanced Analytics\hackathon\group2_data\group2_data_collaborative_filtering.csv', index=False, encoding='utf-8')